# Dask Arrays
     
Dask arrays coordinate many Numpy arrays, arranged into chunks within a grid.  They support a large subset of the Numpy API.

## Numpy-like operations on Dask array

Le's create a 10000x10000 array of random numbers, represented as many numpy arrays of size 1000x1000 (or smaller if the array cannot be divided evenly). In this case there are 100 (10x10) numpy arrays of size 1000x1000.

In [1]:
import dask.array as da
x = da.random.random((10000, 10000), chunks=(1000, 1000))
x

dask.array<random_sample, shape=(10000, 10000), dtype=float64, chunksize=(1000, 1000), chunktype=numpy.ndarray>

Use NumPy syntax as usual

In [ ]:
y = x + x.T
z = y[::2, 5000:].mean(axis=1)
z

Call `.compute()` when you want your result as a NumPy array.

In [ ]:
z.compute()

## Persist data in memory

If you have the available RAM for your dataset then you can persist data in memory.  This allows future computations to be much faster.
Note that this is only relevant if you are in a distributed environment. On a local machine (using single-machine schedulers) `persist` just triggers immediate computation like `compute`.

In [ ]:
y = y.persist()

In [ ]:
%time y[0, 0].compute()

In [ ]:
%time y.sum().compute()

## Stack, Concatenate, and Block

Often we have many arrays stored on disk that we want to stack together and think of as one large array. To solve this problem, we use the functions `da.stack`, `da.concatenate`, and `da.block`.

### Stack
We stack many existing Dask arrays into a new array, creating a new dimension as we go.

In [ ]:
import dask.array as da

arr0 = da.random.random((3, 4), chunks=(1, 2))
arr1 = da.random.random((3, 4), chunks=(1, 2))

data = [arr0, arr1]

In [ ]:
arr0

In [ ]:
arr1

In [ ]:
da.stack(data, axis=0)

In [ ]:
da.stack(data, axis=1)

In [ ]:
da.stack(data, axis=-1)

### Concatenate
We concatenate existing arrays into a new array, extending them along an existing dimension

In [ ]:
import dask.array as da

arr0 = da.random.random((3, 4), chunks=(1, 2))
arr1 = da.random.random((3, 4), chunks=(1, 2))

data = [arr0, arr1]

In [ ]:
da.concatenate(data, axis=0)

In [ ]:
da.concatenate(data, axis=1)

### Block 
We can handle a larger variety of cases with `da.block` as it allows concatenation to be applied over multiple dimensions at once. This is useful if your chunks tile a space, for example if small squares tile a larger 2-D plane..

In [ ]:
import dask.array as da
import numpy as np

arr0 = da.random.random((3, 4), chunks=(1, 2))
arr1 = da.random.random((3, 4), chunks=(1, 2))

data = [
    [arr0, arr1],
    [arr1, arr0]
]

In [ ]:
arr0

In [ ]:
arr1

In [ ]:
da.block(data)

## Get to know the chunks

If you have a Dask array and want to know more information about chunks and their size, you can use the `chunksize` and `chunks` attributes to access this information.

We always specify a chunks argument to tell `dask.array` how to break up the underlying array into chunks. We can specify chunks in a variety of ways: 
- A uniform dimension size like `1000`, meaning chunks of size `1000` in each dimension 
- A uniform chunk shape like `(1000, 2000, 3000)`, meaning chunks of size `1000` in the first axis, `2000` in the second axis, and 3000 in the third 
- Fully explicit sizes of all blocks along all dimensions, like `((1000, 1000, 500), (400, 400), (5, 5, 5, 5, 5))` 
- A dictionary specifying chunk size per dimension like `{0: 1000, 1: 2000, 2: 3000}`. This is just another way of writing the forms 2 and 3 above

Chunks may include three special values:
- `-1` : no chunking along this dimension
- `None` : no change to the chunking along this dimension (useful for rechunk)
- `"auto"` : allow the chunking in this dimension to accommodate ideal chunk sizes

In [ ]:
darr = da.random.random((1000, 1000, 1000))
darr

In [ ]:
darr.chunksize

In [ ]:
darr.chunks

Sometimes you need to change the chunking layout of your data. For example, perhaps it comes to you chunked row-wise, but you need to do an operation that is much faster if done across columns. You can change the chunking with the rechunk method.  sizes

In [ ]:
darr = darr.rechunk([100, None, None])

In [ ]:
darr

In [ ]:
darr.chunksize

In [ ]:
darr.chunks

## Operate with blocks

`dask.array.Array.blocks` offers an array-like interface to the blocks of an array. This returns a Blockview object that provides an array-like interface to the blocks of a dask array. Numpy-style indexing of a Blockview object returns a selection of blocks as a new dask array. You can index `array.blocks` like a numpy array of shape equal to the number of blocks in each dimension, (available as `array.blocks.size`).

In [ ]:
x = da.arange(8, chunks=2)
x

In [ ]:
x.blocks

In [ ]:
x.blocks.size

In [ ]:
x.blocks.shape # aliases x.numblocks

In [ ]:
x.numblocks

In [ ]:
x.blocks[0].compute()

In [ ]:
x.blocks[:3].compute()

In [ ]:
x.blocks[::2].compute()

In [ ]:
x.blocks[[-1, 0]].compute()

## Dask arrays from different sources

Create dask array from something that looks like an array.
Input must have a .shape, .ndim, .dtype and support numpy-style slicing.

In [ ]:
import numpy as np
a = da.from_array(np.array([[1, 2], [3, 4]]), chunks=(1,1))
a

You can create a dask array from a dask delayed value (this routine is useful for constructing dask arrays in an ad-hoc fashion using dask delayed, particularly when combined with stack and concatena).

The dask array will consist of a single chunk.

In [ ]:
import dask.delayed as dd
import dask.array as da
import numpy as np
value = dd(np.ones)(5)
array = da.from_delayed(value, (5,), dtype=float)
array.compute()
array

## Further Reading 

A more in-depth guide to working with Dask arrays can be found in the [dask tutorial](https://tutorial.dask.org/02_array.html).